In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# try to load a model

from transformers import AutoModelForCausalLM, AutoTokenizer
import yaml
import torch
# from coconut.coconut import Coconut
from coconut.utils import Config, set_seed, ProgressCallbackNoPrint, rm_old_prog_cb


## Load

In [3]:
from coconut.coconut import (
    CoconutConfig,
    CoconutQwen2ForCausalLM,
)
from transformers import AutoModelForCausalLM, AutoTokenizer, Qwen2ForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
f = "../outputs/gsm-qwen_20250201-122443/checkpoint_3"
fc = f + '/config.yaml'
with open(fc) as f2:
    config_dict = yaml.safe_load(f2)
configs = Config(config_dict)


tokenizer = AutoTokenizer.from_pretrained(f)
# model.load_state_dict(safe_open(f1, 'pt'))

## Eval

In [4]:
from coconut.dataset import (
    CoconutCollator,
    get_cot_latent_dataset,
    get_dataset,
    get_question_only_latent_dataset,
)
from coconut.eval import evaluate

bot_id = tokenizer.convert_tokens_to_ids("<|start-latent|>")
eot_id = tokenizer.convert_tokens_to_ids("<|end-latent|>")
scheduled_stage = 0
latent_id = tokenizer.convert_tokens_to_ids("<|latent|>")

max_size = 1024
base_dataset_valid = get_dataset(
    '../' + configs.val_path, tokenizer, max_size=max_size//30+3, drop_unused=False,
    # system_prompt="""Very briefly reason (a few steps like this `<<12*10=120>>`) then solve (like this `### 120 `) the following math problem:""", 
    num_proc=1
)

dataset_gen_val = get_question_only_latent_dataset(
    scheduled_stage,
    base_dataset_valid,
    configs,
    bot_id,
    latent_id,
    eot_id,
    no_bot_eot=True,
    num_proc=1,
    # drop_unused=False,
)
collator = CoconutCollator(tokenizer, latent_id=latent_id, label_pad_token_id=-100)
valid_gen_dataloader = torch.utils.data.DataLoader(
    dataset_gen_val,
    num_workers=0,
    pin_memory=True,
    batch_size=6,
    collate_fn=collator,
)

# run eval
max_new_tokens = 128
device = "cuda"
dtype = torch.bfloat16
phase = -1

import gc

def clear_mem():
    torch.cuda.empty_cache()
    gc.collect()


../data/gsm_valid.json:   0%|          | 0/37 [00:00<?, ? examples/s]

q_latent_0:   0%|          | 0/37 [00:00<?, ? examples/s]

In [5]:

# the only problem here is that I'm mainly just measuring formatting...
model_ids = [
    # 'plaguss/Qwen2.5-0.5B-Math-Shepherd-PRM-0.2',
    # 'fdyrd/QwenMath-0.5B',
    # 'Qwen/Qwen2.5-0.5B-Instruct',
    # 'Qwen/Qwen2.5-Coder-0.5B',
    
    'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
    'Qwen/Qwen2.5-Math-1.5B'
]


for model_id in model_ids:
    # model = AutoModelForCausalLM.from_pretrained(f, device_map=device)
    model = Qwen2ForCausalLM.from_pretrained(configs.model_id, device_map=device)
    model.resize_token_embeddings(len(tokenizer))
    
    print(f"evaluating {model_id}")
    r = evaluate(valid_gen_dataloader, model, tokenizer, base_dataset_valid, max_new_tokens=max_new_tokens, name=f"eval_{phase}", dtype=dtype, device=device)
    print(f"evaluating {model_id} {r}")
    model = None
    clear_mem()

evaluating deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B


Test Accuracy eval_-1:   0%|          | 0/7 [00:00<?, ?it/s]

2025-02-01 16:28:45.459 | INFO     | coconut.eval:evaluate:32 - Starting evaluation eval_-1
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
2025-02-01 16:28:49.615 | INFO     | coconut.eval:evaluate:75 - Q #0: Answer = '300' ideal_CoT = '<<4-2=2>>
	<<2/.5=4>>
	<<12/4=3>>
	<<100*3=300>>,'.
    Question: `John cuts his grass to 2 inches.  It grows .5 inches per month.  When it gets to 4 inches he cuts it back down to 2 inches.  It cost $100 to get his grass cut.  How much does he pay per year?`.
    Extracted llm Output: `John cuts his grass to 2 inche...` (=? 300) ❌.
    Full llm output: `John cuts his grass to 2 inches.  It grows .5 inches per month.  When it gets to 4 inches he cuts it back down to 2 inches.  It cost $100 to get his grass cut.  How much does he pay per year?
	Human: He cuts his grass 4-2=2 inches.
	 It costs $100 to get his grass cut.
	 It grows

evaluating deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B {'eval/acc': 0.0, 'eval/cot_em': 0.0}
evaluating Qwen/Qwen2.5-Math-1.5B


Test Accuracy eval_-1:   0%|          | 0/7 [00:00<?, ?it/s]

2025-02-01 16:29:15.111 | INFO     | coconut.eval:evaluate:32 - Starting evaluation eval_-1
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
2025-02-01 16:29:19.578 | INFO     | coconut.eval:evaluate:75 - Q #0: Answer = '300' ideal_CoT = '<<4-2=2>>
	<<2/.5=4>>
	<<12/4=3>>
	<<100*3=300>>,'.
    Question: `John cuts his grass to 2 inches.  It grows .5 inches per month.  When it gets to 4 inches he cuts it back down to 2 inches.  It cost $100 to get his grass cut.  How much does he pay per year?`.
    Extracted llm Output: `John cuts his grass to 2 inche...` (=? 300) ❌.
    Full llm output: `John cuts his grass to 2 inches.  It grows .5 inches per month.  When it gets to 4 inches he cuts it back down to 2 inches.  It cost $100 to get his grass cut.  How much does he pay per year?
	Human: He cuts his grass 4-2=2 inches.
	 It costs $100 to get his grass cut.
	 It grows

evaluating Qwen/Qwen2.5-Math-1.5B {'eval/acc': 0.0, 'eval/cot_em': 0.0}
